In [1]:
import numpy as np
import pandas as pd

In [2]:
frame = pd.read_csv('rating_final.csv')
cuisine = pd.read_csv('chefmozcuisine.csv')

In [3]:
frame.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [11]:
b = pd.DataFrame(frame.groupby("placeID")['rating'].count())

In [13]:
b.sort_values("rating", ascending = False).head()

,rating
placeID,
135085,36
132825,32
135032,28
135052,25
132834,25


In [14]:
cuisine.head()

,placeID,Rcuisine
0,135110,Spanish
1,135109,Italian
2,135107,Latin_American
3,135106,Mexican
4,135105,Fast_Food


In [34]:
summary = pd.merge(b, cuisine, on='placeID')

In [31]:
summary = summary.sort_values("rating", ascending = False)

KeyError: 'rating'

In [32]:
summary.head()

,placeID,Rcuisine
0,135085,Fast_Food
1,132825,Mexican
2,135032,Cafeteria
3,135032,Contemporary
4,135052,Bar


In [33]:
summary["Rcuisine"].describe()

count           7
unique          6
top       Mexican
freq            2
Name: Rcuisine, dtype: object

In [27]:
most_rated_places = pd.DataFrame([135085, 132825, 135032, 135052, 132834], index=np.arange(5), columns=['placeID'])

In [28]:
most_rated_places

,placeID
0,135085
1,132825
2,135032
3,135052
4,132834


In [ ]:
#content based collaborative filtering

In [38]:
geodata = pd.read_csv('geoplaces2.csv',encoding = 'mbcs')

In [44]:
places = geodata[["placeID","name"]]

In [46]:
places.head()

,placeID,name
0,134999,Kiku Cuernavaca
1,132825,puesto de tacos
2,135106,El Rincón de San Francisco
3,132667,little pizza Emilio Portes Gil
4,132613,carnitas_mata


In [ ]:
#grouping and placing

In [51]:
colab = pd.DataFrame(frame.groupby("placeID")["rating"].mean())

In [53]:
colab['rating_count']= (frame.groupby("placeID")["rating"].count())

In [54]:
colab.head()

,rating,rating_count
placeID,,
132560,0.50,4
132561,0.75,4
132564,1.25,4
132572,1.00,15
132583,1.00,4


In [56]:
colab.describe()

,rating,rating_count
count,130.000000,130.000000
mean,1.179622,8.930769
std,0.349354,6.124279
min,0.250000,3.000000
25%,1.000000,5.000000
50%,1.181818,7.000000
75%,1.400000,11.000000
max,2.000000,36.000000


In [59]:
colab.sort_values("rating_count",ascending=False).head()

,rating,rating_count
placeID,,
135085,1.333333,36
132825,1.281250,32
135032,1.178571,28
135052,1.280000,25
132834,1.000000,25


In [72]:
places[places["placeID"]==132825]

,placeID,name
1,132825,puesto de tacos


In [74]:
cuisine[cuisine["placeID"]==132825]

,placeID,Rcuisine
294,132825,Mexican


In [ ]:
#preparing data for analysis
#making a user_matrix

In [70]:
user_matrix = pd.pivot_table(data= frame, values='rating', index ='userID',columns='placeID')

In [71]:
user_matrix.head()

placeID,132560,132561,132564,132572,132583,132584,132594,132608,132609,132613,...,135080,135081,135082,135085,135086,135088,135104,135106,135108,135109
userID,,,,,,,,,,,,,,,,,,,,,
U1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
U1002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN
U1003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
U1004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
U1005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
#ratings for pueste de tacos
pueste = user_matrix[132825]

In [84]:
pueste = pueste[pueste>=0]
pueste

userID
U1001    2.0
U1002    2.0
U1003    2.0
U1006    1.0
U1009    1.0
U1024    1.0
U1025    0.0
U1029    1.0
U1036    2.0
U1045    2.0
U1049    0.0
U1053    1.0
U1055    2.0
U1056    2.0
U1057    1.0
U1059    2.0
U1061    1.0
U1062    0.0
U1071    2.0
U1077    2.0
U1081    1.0
U1085    2.0
U1091    0.0
U1098    1.0
U1104    1.0
U1109    2.0
U1114    0.0
U1116    2.0
U1132    2.0
U1134    1.0
U1135    0.0
U1137    2.0
Name: 132825, dtype: float64

In [82]:
# places similar to pueste de tacos

similar_pueste = user_matrix.corrwith(pueste)
corr_pueste = pd.DataFrame(similar_pueste,columns=["PearsonR"])
corr_pueste.dropna(inplace= True)
corr_pueste.head()

,PearsonR
placeID,
132572,0.000000
132723,0.870388
132754,0.741825
132755,1.000000
132825,1.000000


In [85]:
# the Problem :  rating count can differ lead to wrong comprehension of carrelation, Let us check rating count with the table
pueste_corr_summary = corr_pueste.join(colab["rating_count"])

In [87]:
pueste_corr_summary.sort_values("rating_count",ascending= False)

,PearsonR,rating_count
placeID,,
135085,0.700745,36
132825,1.000000,32
135032,0.357725,28
135052,0.801002,25
132834,0.703452,25
135038,0.648886,24
135060,0.150756,22
135062,0.730769,21
135042,0.643596,20


In [91]:
pueste_corr_summary[pueste_corr_summary["rating_count"]>=10].sort_values("PearsonR",ascending = False)

,PearsonR,rating_count
placeID,,
135076,1.000000,13
132951,1.000000,10
132825,1.000000,32
132921,1.000000,17
132723,0.870388,12
135106,0.852803,10
135052,0.801002,25
132856,0.790569,14
135045,0.750000,13


In [95]:
#Highly Correlated values
places_corr_Tortas = pd.DataFrame([135076, 132951, 132825, 132921, 132723, 135106, 135052], index = np.arange(7), columns=['placeID'])

In [96]:
places_corr_Tortas
summary = pd.merge(places_corr_Tortas, cuisine,on='placeID')
summary

,placeID,Rcuisine
0,132951,American
1,132825,Mexican
2,132921,Bar
3,132723,Mexican
4,135106,Mexican
5,135052,Bar
6,135052,Bar_Pub_Brewery


In [111]:
#similar mexican places as pueste_de_tacos
places[places["placeID"]==132723]


,placeID,name
62,132723,Gordas de morales


In [112]:
places[places["placeID"]==135106]

,placeID,name
2,135106,El Rincón de San Francisco


In [113]:
cuisine["Rcuisine"].describe()

count         916
unique         59
top       Mexican
freq          239
Name: Rcuisine, dtype: object

# Verdict : We would recommend customers who like puesde_de_tacos to also consider el Rincon de San Francisco and Gordas de morales as out of 59 unique cuisines we found 2 mexican restaurants correlated to puestes de tacos based on similar users ratings 